# Área de imports

In [1]:
import torch
import pandas as pd
import texthero as hero
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel

In [2]:
# Variável que indica a disponibilidade de uma GPU com CUDA na máquina / ambiente
cuda_available = torch.cuda.is_available()

# Carregamento dos dados

In [3]:
df = pd.read_csv('./data/SMSSpamCollection.txt', sep='\t', names=["labels", "text"])

# Modelagem

#### Atribuindo um valor numérico para os labels

In [4]:
df['labels'] = [1 if row == 'spam' else 0 for row in df['labels']]

#### Extraindo uma amostra pequena dos dados para acelerar o treinamento

In [5]:
df = df.sample(n=500, random_state=1)

#### Dividindo o conjunto de dados entre treino e teste

In [6]:
train, test = train_test_split(df, test_size=0.3)

#### Pré-processamento do texto

In [7]:
custom_pipeline = [hero.preprocessing.lowercase,
                   hero.preprocessing.remove_digits,
                   hero.preprocessing.remove_punctuation,
                   hero.remove_stopwords]

df['text'] = hero.clean(df["text"], custom_pipeline)

#### Instanciando o modelo

Observação importante: 
- A utilização de uma GPU com CUDA habilitado é fortemente indicada para esta parte do tutorial e para a realização do projeto
- A ferramenta online [Google Colaboratory](https://colab.research.google.com/) disponibiliza gratuitamente um ambiente com GPU + CUDA para utilização em notebooks
- Há um tutorial detalhado de como utilizar o Google Colab no repositório deste tutorial

In [8]:
model = ClassificationModel(
    "roberta", 
    "roberta-base",
    use_cuda=cuda_available
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.

#### Treinando o modelo

In [9]:
model.train_model(train)

  0%|          | 0/350 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/44 [00:00<?, ?it/s]

(44, 0.25328544780230994)

#### Avaliando o modelo

In [12]:
result, model_outputs, wrong_predictions = model.eval_model(test)

  0%|          | 0/150 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/19 [00:00<?, ?it/s]

#### Visualizando as métricas de avaliação

In [14]:
result

{'mcc': 0.9034480357884764,
 'tp': 16,
 'tn': 131,
 'fp': 1,
 'fn': 2,
 'auroc': 0.9991582491582491,
 'auprc': 0.9939896036387266,
 'eval_loss': 0.04893595976509938}

#### Fazendo predições com o modelo treinado

In [33]:
# Tabela de correspondência
cor_tab = {0: 'ham', 1: 'spam'}

In [35]:
# Fazendo a predição a partir de uma mensagem
predictions, raw_outputs = model.predict(["URGENT! We are trying to contact you. Last weekends draw shows that you have won a £900 prize GUARANTEED. Call 09061701851. Claim code K61. Valid 12hours only"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [36]:
cor_tab[predictions[0]]

'spam'